In [1]:
import pandas as pd
import os

In [3]:
# Define paths and load data
input_path = "C:/Users/Skyline/Time Series Forecasting/data/processed"
df_tsla = pd.read_csv(os.path.join(input_path, "TSLA_cleaned.csv"))

# Convert Date to datetime and set as index
df_tsla['Date'] = pd.to_datetime(df_tsla['Date'])
df_tsla.set_index('Date', inplace=True)

# Split into training (2015-2023) and testing (2024-2025) sets
train = df_tsla['Close'].loc['2015-07-01':'2023-12-31']
test = df_tsla['Close'].loc['2024-01-01':'2025-07-31']

print(f"Train size: {len(train)}, Test size: {len(test)}")

Train size: 2140, Test size: 396
